In [1]:
import numpy as np
import pandas as pd
import import_ipynb
import als

In [2]:
def recommend_anime(R, 
                    u_id,
                    original_df = pd.read_csv("data/100x100.csv"), 
                    x=5):
    
    original_matrix = original_df.pivot(index='u_id', columns='a_id', values='score').fillna(0)
    R_df = pd.DataFrame(R, index=original_matrix.index, columns=original_matrix.columns)
    
    # find the user row of original_matrix
    user_row = original_matrix.loc[u_id]
    
    # get the anime IDs that the user has already watched (ratings > 0)
    user_watched = user_row[user_row > 0].index.tolist()
    
    # get the anime IDs that the user has not watched (ratings == 0)
    user_not_watched = user_row[user_row == 0].index.tolist()
    
    # get predicted ratings for the unwatched anime 
    user_pred = R_df.loc[u_id, user_not_watched]
    # R_df contains the predicted ratings for all users and all anime id, but not anime names
    # find top x anime ids with the highest predicted ratings that the user has not watched, and sort them by predicted rating
    # then return the anime names and predicted ratings
    top_x = user_pred.sort_values(ascending=False).head(x)
    anime_ids = top_x.index
    # get the anime names from original_df
    anime_names = original_df[original_df['a_id'].isin(anime_ids)]['title'].unique()
    # create a DataFrame with anime names and predicted ratings
    recommendations = pd.DataFrame({
        'title': anime_names,
        'predicted_rating': top_x.values
    })
    
    return recommendations

In [3]:
original_df = pd.read_csv("data/100x100.csv")
original_matrix = original_df.pivot(index='u_id', columns='a_id', values='score').fillna(0)
X, Y = als.als(original_matrix.values, 20, 100)
R = X @ Y.T
recommend_anime(R, 966, original_df)

,title,predicted_rating
0,Ano Hi Mita Hana no Namae wo Bokutachi wa Mada...,9.311181
1,Toradora!,8.861783
2,FLCL,8.805558
3,Lucky☆Star,8.792845
4,Nichijou,8.563679
